# Exploratory Data Analysis using Python - A Case Study
Analyzing responses from the Stack Overflow Annual Developer Survey 2020 (the 2021 survey is not compatible this time)

## Importing Libraries
Read the CSV file too

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

survey_raw_df = pd.read_csv("2020survey_results_public.csv")
schema_fname = "2020survey_results_schema.csv"
schema_raw = pd.read_csv(schema_fname, index_col="Column")["QuestionText"]

print(survey_raw_df.columns)
print(schema_raw["YearsCodePro"])

Index(['Respondent', 'MainBranch', 'Hobbyist', 'Age', 'Age1stCode', 'CompFreq',
       'CompTotal', 'ConvertedComp', 'Country', 'CurrencyDesc',
       'CurrencySymbol', 'DatabaseDesireNextYear', 'DatabaseWorkedWith',
       'DevType', 'EdLevel', 'Employment', 'Ethnicity', 'Gender', 'JobFactors',
       'JobSat', 'JobSeek', 'LanguageDesireNextYear', 'LanguageWorkedWith',
       'MiscTechDesireNextYear', 'MiscTechWorkedWith',
       'NEWCollabToolsDesireNextYear', 'NEWCollabToolsWorkedWith', 'NEWDevOps',
       'NEWDevOpsImpt', 'NEWEdImpt', 'NEWJobHunt', 'NEWJobHuntResearch',
       'NEWLearn', 'NEWOffTopic', 'NEWOnboardGood', 'NEWOtherComms',
       'NEWOvertime', 'NEWPurchaseResearch', 'NEWPurpleLink', 'NEWSOSites',
       'NEWStuck', 'OpSys', 'OrgSize', 'PlatformDesireNextYear',
       'PlatformWorkedWith', 'PurchaseWhat', 'Sexuality', 'SOAccount',
       'SOComm', 'SOPartFreq', 'SOVisitFreq', 'SurveyEase', 'SurveyLength',
       'Trans', 'UndergradMajor', 'WebframeDesireNextYear',
  

## Data Preparation & Cleaning
### Select a subset of columns with the relevant data for our analysis.
While the survey responses contain a wealth of information, we'll limit our analysis to the following areas:
* Demographics of the survey respondents and the global programming community
* Distribution of programming skills, experience, and preferences
* Employment-related information, preferences, and opinions

In [3]:
selected_columns = [
    # Demographics
    'Country',
    'Age',
    'Gender',
    'EdLevel',
    'UndergradMajor',
    # Programming experience
    'Hobbyist',
    'Age1stCode',
    'YearsCode',
    'YearsCodePro',
    'LanguageWorkedWith',
    'LanguageDesireNextYear',
    'NEWLearn',
    'NEWStuck',
    # Employment
    'Employment',
    'DevType',
    'WorkWeekHrs',
    'JobSat',
    'JobFactors',
    'NEWOvertime',
    'NEWEdImpt'
]

print(len(selected_columns))
print(selected_columns)

20
['Country', 'Age', 'Gender', 'EdLevel', 'UndergradMajor', 'Hobbyist', 'Age1stCode', 'YearsCode', 'YearsCodePro', 'LanguageWorkedWith', 'LanguageDesireNextYear', 'NEWLearn', 'NEWStuck', 'Employment', 'DevType', 'WorkWeekHrs', 'JobSat', 'JobFactors', 'NEWOvertime', 'NEWEdImpt']


### Extract a copy of the data from these columns into a new data frame.
We can continue to modify further without affecting the original data frame.

In [4]:
survey_df = survey_raw_df[selected_columns].copy()
schema = schema_raw[selected_columns]

print(survey_df.shape)
print(survey_df.info())

(64461, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64461 entries, 0 to 64460
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 64072 non-null  object 
 1   Age                     45446 non-null  float64
 2   Gender                  50557 non-null  object 
 3   EdLevel                 57431 non-null  object 
 4   UndergradMajor          50995 non-null  object 
 5   Hobbyist                64416 non-null  object 
 6   Age1stCode              57900 non-null  object 
 7   YearsCode               57684 non-null  object 
 8   YearsCodePro            46349 non-null  object 
 9   LanguageWorkedWith      57378 non-null  object 
 10  LanguageDesireNextYear  54113 non-null  object 
 11  NEWLearn                56156 non-null  object 
 12  NEWStuck                54983 non-null  object 
 13  Employment              63854 non-null  object 
 14  DevType                 49

### Convert some other columns into numeric data types while ignoring any non-numeric value.
Most columns have the data type object, either because they contain values of different types or contain empty values (NaN). It appears that every column contains some empty values since the Non-Null count for every column is lower than the total number of rows (64461). We'll need to deal with empty values and manually adjust the data type for each column on a case-by-case basis. Only two of the columns were detected as numeric columns (Age and WorkWeekHrs), even though a few other columns have mostly numeric values. The non-numeric are converted to NaN.

In [5]:
survey_df['Age1stCode'] = pd.to_numeric(survey_df.Age1stCode, errors='coerce')
survey_df['YearsCode'] = pd.to_numeric(survey_df.YearsCode, errors='coerce')
survey_df['YearsCodePro'] = pd.to_numeric(survey_df.YearsCodePro, errors='coerce')

print(survey_df.describe())

                Age    Age1stCode     YearsCode  YearsCodePro   WorkWeekHrs
count  45446.000000  57473.000000  56784.000000  44133.000000  41151.000000
mean      30.834111     15.476572     12.782051      8.869667     40.782174
std        9.585392      5.114081      9.490657      7.759961     17.816383
min        1.000000      5.000000      1.000000      1.000000      1.000000
25%       24.000000     12.000000      6.000000      3.000000     40.000000
50%       29.000000     15.000000     10.000000      6.000000     40.000000
75%       35.000000     18.000000     17.000000     12.000000     44.000000
max      279.000000     85.000000     50.000000     50.000000    475.000000


There seems to be a problem with the age column, as the minimum value is 1 and the maximum is 279. This is a common issue with surveys: responses may contain invalid values due to accidental or intentional errors while responding. A simple fix would be to ignore the rows where the age is higher than 100 years or lower than 10 years as invalid survey responses. We can do this using the .drop method, as explained here.

### Change the invalid data to the valid one

In [6]:
survey_df.drop(survey_df[survey_df.Age < 10].index, inplace=True)
survey_df.drop(survey_df[survey_df.Age > 100].index, inplace=True)

# ignore entries where the value for the column is higher than 140 hours. (~20 hours per day)
survey_df.drop(survey_df[survey_df.WorkWeekHrs > 140].index, inplace=True)

The gender column also allows for picking multiple options. We'll remove values containing more than one option to simplify our analysis.

In [13]:
print(survey_df['Gender'].value_counts())

Man                                                  45895
Woman                                                 3835
Non-binary, genderqueer, or gender non-conforming      385
Name: Gender, dtype: int64


In [14]:
survey_df.where(~(survey_df.Gender.str.contains(';', na=False)), np.nan, inplace=True)

# We've now cleaned up and prepared the dataset for analysis
survey_df.sample(10)

,Country,Age,Gender,EdLevel,UndergradMajor,Hobbyist,Age1stCode,YearsCode,YearsCodePro,LanguageWorkedWith,LanguageDesireNextYear,NEWLearn,NEWStuck,Employment,DevType,WorkWeekHrs,JobSat,JobFactors,NEWOvertime,NEWEdImpt
20955,Thailand,29.0,Man,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",Yes,11.0,15.0,5.0,Bash/Shell/PowerShell;C#;HTML/CSS;JavaScript;K...,JavaScript;Kotlin;Rust;TypeScript,Every few months,Call a coworker or friend;Visit Stack Overflow...,Employed full-time,"Developer, full-stack;DevOps specialist",40.0,Slightly satisfied,Industry that I’d be working in;Financial perf...,Rarely: 1-2 days per year or less,Not at all important/not necessary
43294,China,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,C;HTML/CSS;Java,HTML/CSS;Java;JavaScript;Kotlin,Once a year,Meditate;Play games;Call a coworker or friend;...,Student,NaN,NaN,NaN,NaN,NaN,NaN
10400,Spain,44.0,Man,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Computer science, computer engineering, or sof...",No,12.0,25.0,19.0,Java;Objective-C;Swift,JavaScript;TypeScript,Once every few years,Call a coworker or friend;Visit Stack Overflow...,Employed full-time,"Developer, back-end;Developer, mobile",NaN,Slightly dissatisfied,"Flex time or a flexible schedule;Languages, fr...",Rarely: 1-2 days per year or less,Very important
12378,France,24.0,Man,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",Yes,10.0,15.0,3.0,Bash/Shell/PowerShell;C++;Go;Rust,C++;Rust,Once every few years,Call a coworker or friend;Visit Stack Overflow...,Employed full-time,"Academic researcher;Developer, game or graphics",37.0,Slightly dissatisfied,Industry that I’d be working in;Flex time or a...,Sometimes: 1-2 days per month but less than we...,Fairly important
5334,Sri Lanka,48.0,Man,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",I never declared a major,Yes,13.0,34.0,29.0,Assembly;C;C++;HTML/CSS;Java;JavaScript;PHP;Py...,Assembly;C;C++;HTML/CSS;JavaScript;PHP;Python;SQL,Once every few years,Visit Stack Overflow;Watch help / tutorial vid...,"Independent contractor, freelancer, or self-em...",Academic researcher;Database administrator;Dev...,10.0,Very satisfied,Industry that I’d be working in;Opportunities ...,Often: 1-2 days per week or more,Somewhat important
61754,India,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Assembly;C;C++;HTML/CSS;Java;PHP;Python;Rust,Assembly;C;C++;Python;Rust;SQL,Every few months,NaN,Student,NaN,NaN,NaN,NaN,NaN,NaN
21189,India,NaN,Man,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Computer science, computer engineering, or sof...",No,19.0,9.0,6.0,Java;JavaScript,Java;Python,Every few months,Call a coworker or friend;Visit Stack Overflow...,Employed full-time,"Developer, back-end;Product manager",35.0,Very satisfied,Flex time or a flexible schedule;Specific depa...,Sometimes: 1-2 days per month but less than we...,Very important
61558,United States,36.0,Man,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","A humanities discipline (such as literature, h...",Yes,9.0,9.0,9.0,C#;HTML/CSS;JavaScript;Python;SQL;TypeScript,C#;HTML/CSS;SQL;TypeScript,Once every few years,Visit Stack Overflow;Do other work and come ba...,Employed full-time,"Developer, back-end;Developer, front-end;Devel...",36.0,Very satisfied,Industry that I’d be working in;Flex time or a...,Occasionally: 1-2 days per quarter but less th...,Fairly important
53008,United States,20.0,Man,NaN,"Computer science, computer engineering, or sof...",Yes,9.0,11.0,NaN,HTML/CSS;Java;JavaScript;PHP;Python;SQL;TypeSc...,C#;Dart;HTML/CSS;JavaScript;Python;Ruby;Rust;S...,Every few months,Play games;Call a coworker or friend;Visit Sta...,Student,NaN,NaN,NaN,Flex time or a flexible schedule;Remote work o...,NaN,NaN
49489,India,29.0,Man,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",Yes,19.0,10.0,5.0,C;C#;C++;Dart;HTML/CSS;Java;JavaScript;Python,HTML/CSS;Java;JavaScript;Python,Once every few years

## Exploratory Analysis and Visualization